In [2]:
pip install --upgrade "transformers>=4.37" datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [5]:
from datasets import load_dataset
from transformers import (
    LukeTokenizer,
    LukeForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import evaluate

In [4]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
pip install huggingface_hub


In [ ]:
from huggingface_hub import login
login()

In [6]:
ds = load_dataset("ailsntua/QEvasion")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

In [7]:
ds = ds.filter(lambda x: x["interview_question"] is not None and x["interview_answer"] is not None)

Filter:   0%|          | 0/3448 [00:00<?, ? examples/s]

Filter:   0%|          | 0/308 [00:00<?, ? examples/s]

In [8]:
evasion_labels = sorted(set(ds["train"]["evasion_label"]))
evasion2id = {c: i for i, c in enumerate(evasion_labels)}
id2evasion = {i: c for c, i in evasion2id.items()}
num_labels = len(evasion_labels)

print(evasion2id)

{'Claims ignorance': 0, 'Clarification': 1, 'Declining to answer': 2, 'Deflection': 3, 'Dodging': 4, 'Explicit': 5, 'General': 6, 'Implicit': 7, 'Partial/half-answer': 8}


In [9]:
train_valid = ds["train"].train_test_split(test_size=0.15, seed=42)
train_ds = train_valid["train"]
valid_ds = train_valid["test"]

print("Train size:", len(train_ds))
print("Valid size:", len(valid_ds))
print("HF test size (do NOT train on this):", len(ds["test"]))

Train size: 2930
Valid size: 518
HF test size (do NOT train on this): 308


In [10]:
from transformers import LukeTokenizer, LukeForSequenceClassification
model_name = "studio-ousia/luke-base"

tokenizer = LukeTokenizer.from_pretrained(model_name)

model = LukeForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2evasion,
    label2id=evasion2id,
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

Some weights of LukeForSequenceClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def preprocess(batch):
    qs = [q or "" for q in batch["question"]]
    ans = [a or "" for a in batch["interview_answer"]]

    texts = [f"Q: {q} [SEP] A: {a}" for q, a in zip(qs, ans)]

    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    enc["labels"] = [evasion2id[label] for label in batch["evasion_label"]]
    return enc


In [12]:
train_encoded = train_ds.map(preprocess, batched=True)
valid_encoded = valid_ds.map(preprocess, batched=True)

cols = ["input_ids", "attention_mask", "labels"]
train_encoded = train_encoded.remove_columns([c for c in train_encoded.column_names if c not in cols])
valid_encoded = valid_encoded.remove_columns([c for c in valid_encoded.column_names if c not in cols])

train_encoded.set_format("torch")
valid_encoded.set_format("torch")


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

Map:   0%|          | 0/2930 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [13]:
from collections import Counter
import torch
import torch.nn as nn
label_list_train = train_ds["evasion_label"]
counts = Counter(label_list_train)
print("Train label counts:", counts)

class_counts = np.array([counts[label] for label in evasion_labels], dtype=float)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_weights)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print("Class weights:", class_weights)

loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor.to(model.device))

Train label counts: Counter({'Explicit': 908, 'Dodging': 585, 'Implicit': 398, 'Deflection': 336, 'General': 325, 'Declining to answer': 124, 'Claims ignorance': 105, 'Clarification': 78, 'Partial/half-answer': 71})
Class weights: [1.53418072 2.06524327 1.29910464 0.47943147 0.27536577 0.17741077
 0.49565839 0.40474617 2.26885881]


In [14]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "macro_f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

In [23]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Ensure class_weights_tensor is on the same device as the model
        loss = nn.CrossEntropyLoss(weight=class_weights_tensor.to(model.device))(logits, labels)
        if return_outputs:
            return loss, outputs
        return loss

training_args = TrainingArguments(
    output_dir="./luke-qevasion",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    report_to="none",
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=valid_encoded,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.327000,2.002756,0.403475,0.349451
2,1.052300,1.898085,0.376448,0.384923
3,0.697600,2.169678,0.388031,0.375109
4,0.453300,2.218036,0.401544,0.378054


TrainOutput(global_step=1468, training_loss=0.8725914760246589, metrics={'train_runtime': 172.0736, 'train_samples_per_second': 68.11, 'train_steps_per_second': 8.531, 'total_flos': 1928239098900480.0, 'train_loss': 0.8725914760246589, 'epoch': 4.0})

In [24]:
test_ds = load_dataset("ailsntua/QEvasion", split="test")

In [25]:
def preprocess_test(batch):
    qs = [q or "" for q in batch["question"]]
    ans = [a or "" for a in batch["interview_answer"]]
    texts = [f"Q: {q} [SEP] A: {a}" for q, a in zip(qs, ans)]

    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    return enc

test_encoded = test_ds.map(preprocess_test, batched=True)
test_encoded.set_format(type="torch", columns=["input_ids", "attention_mask"])

pred_output = trainer.predict(test_encoded)
logits = pred_output.predictions
pred_ids = np.argmax(logits, axis=-1)

pred_labels = [id2evasion[int(i)] for i in pred_ids]

print(pred_labels[:10])
print("Total predictions:", len(pred_labels))



['Dodging', 'General', 'Implicit', 'General', 'Partial/half-answer', 'Dodging', 'General', 'Implicit', 'General', 'General']
Total predictions: 308


In [26]:
import numpy as np

pred_output = trainer.predict(test_encoded)
logits = pred_output.predictions
pred_ids = np.argmax(logits, axis=-1)

pred_labels = [id2evasion[int(i)] for i in pred_ids]


In [27]:
with open("prediction", "w", encoding="utf-8") as f:
    for label in pred_labels:
        f.write(str(label).strip() + "\n")


with open("prediction", "r", encoding="utf-8") as f:
    lines = f.readlines()
print("Lines in prediction file:", len(lines))


Lines in prediction file: 308


In [28]:
!zip submission.zip prediction
!unzip -l submission.zip


updating: prediction (deflated 88%)
Archive:  submission.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
     3021  2025-12-09 20:27   prediction
---------                     -------
     3021                     1 file


In [29]:
from google.colab import files
files.download("submission.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import evaluate
import torch
import torch.nn as nn
from collections import Counter

In [ ]:
clarity_labels = sorted(set(ds["train"]["clarity_label"]))
clarity2id = {c: i for i, c in enumerate(clarity_labels)}
id2clarity = {i: c for c, i in clarity2id.items()}
num_clarity = len(clarity_labels)

print("Clarity mapping:", clarity2id)

Clarity mapping: {'Ambivalent': 0, 'Clear Non-Reply': 1, 'Clear Reply': 2}


In [ ]:
train_valid = ds["train"].train_test_split(test_size=0.15, seed=42)
train_ds = train_valid["train"]
valid_ds = train_valid["test"]

print("Train size:", len(train_ds))
print("Valid size:", len(valid_ds))
print("HF test size (not used for training):", len(ds["test"]))

Train size: 2930
Valid size: 518
HF test size (not used for training): 308


In [ ]:
model_name = "microsoft/deberta-v3-base"

clarity_tokenizer = AutoTokenizer.from_pretrained(model_name)

clarity_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_clarity,
    id2label=id2clarity,
    label2id=clarity2id,
)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def make_text(q, a, ev_label):
    # You can tweak wording if you like
    return f"Evasion: {ev_label}. Q: {q} [SEP] A: {a}"

def preprocess_clarity_train(batch):
    qs   = [q or "" for q in batch["question"]]
    ans  = [a or "" for a in batch["interview_answer"]]
    evas = [e or "" for e in batch["evasion_label"]]   # GOLD evasion

    texts = [make_text(q, a, e) for q, a, e in zip(qs, ans, evas)]

    enc = clarity_tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    enc["labels"] = [clarity2id[l] for l in batch["clarity_label"]]
    return enc

train_encoded = train_ds.map(preprocess_clarity_train, batched=True)
valid_encoded = valid_ds.map(preprocess_clarity_train, batched=True)

cols = ["input_ids", "attention_mask", "labels"]
train_encoded = train_encoded.remove_columns(
    [c for c in train_encoded.column_names if c not in cols]
)
valid_encoded = valid_encoded.remove_columns(
    [c for c in valid_encoded.column_names if c not in cols]
)

train_encoded.set_format("torch")
valid_encoded.set_format("torch")


Map:   0%|          | 0/2930 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
from collections import Counter

clr_counts = Counter(train_ds["clarity_label"])
print("Train clarity counts:", clr_counts)

class_counts = np.array([clr_counts[l] for l in clarity_labels], dtype=float)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_weights)
class_weights_t = torch.tensor(class_weights, dtype=torch.float)

print("Clarity class weights:", class_weights)

Train clarity counts: Counter({'Ambivalent': 1715, 'Clear Reply': 908, 'Clear Non-Reply': 307})
Clarity class weights: [0.35397872 1.97743813 0.66858315]


In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "macro_f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

class WeightedClarityTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = nn.CrossEntropyLoss(weight=class_weights_t.to(model.device))(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./deberta-clarity-with-evasion",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    report_to="none",
)

clarity_trainer = WeightedClarityTrainer(
    model=clarity_model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=valid_encoded,
    processing_class=clarity_tokenizer,
    compute_metrics=compute_metrics,
)

clarity_trainer.train()
print("Validation:", clarity_trainer.evaluate())

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
test_ds = ds["test"]

def preprocess_evasion_test(batch):
    qs  = [q or "" for q in batch["question"]]
    ans = [a or "" for a in batch["interview_answer"]]
    texts = [f"Q: {q} [SEP] A: {a}" for q, a in zip(qs, ans)]

    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    return enc

evasion_test_encoded = test_ds.map(preprocess_evasion_test, batched=True)
evasion_test_encoded.set_format(type="torch", columns=["input_ids", "attention_mask"])

# LUKE predictions
evasion_pred_output = trainer.predict(evasion_test_encoded)
evasion_logits = evasion_pred_output.predictions
evasion_pred_ids = np.argmax(evasion_logits, axis=-1)
evasion_pred_labels = [id2evasion[int(i)] for i in evasion_pred_ids]

print(evasion_pred_labels[:10])
print("Total evasion preds:", len(evasion_pred_labels))

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

['Dodging', 'General', 'Implicit', 'General', 'Explicit', 'Implicit', 'Implicit', 'Implicit', 'General', 'Implicit']
Total evasion preds: 308


In [ ]:

assert len(test_ds) == len(evasion_pred_labels)
test_with_evasion = test_ds.add_column("pred_evasion_label", evasion_pred_labels)

def preprocess_clarity_test(batch):
    qs   = [q or "" for q in batch["question"]]
    ans  = [a or "" for a in batch["interview_answer"]]
    evas = [e or "" for e in batch["pred_evasion_label"]]

    texts = [make_text(q, a, e) for q, a, e in zip(qs, ans, evas)]

    enc = clarity_tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    return enc

test_encoded_clarity = test_with_evasion.map(preprocess_clarity_test, batched=True)
test_encoded_clarity.set_format(type="torch", columns=["input_ids", "attention_mask"])



Flattening the indices:   0%|          | 0/308 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [ ]:
clarity_pred_output = clarity_trainer.predict(test_encoded_clarity)
clarity_logits = clarity_pred_output.predictions
clarity_pred_ids = np.argmax(clarity_logits, axis=-1)
clarity_pred_labels = [id2clarity[int(i)] for i in clarity_pred_ids]

print(clarity_pred_labels[:10])
print("Total clarity preds:", len(clarity_pred_labels)
with open("prediction", "w", encoding="utf-8") as f:
    for label in clarity_pred_labels:
        f.write(str(label).strip() + "\n")

with open("prediction", "r", encoding="utf-8") as f:
    lines = f.readlines()
print("Lines in prediction file:", len(lines))

!zip submission.zip prediction
!unzip -l submission.zip


['Ambivalent', 'Ambivalent', 'Ambivalent', 'Ambivalent', 'Clear Reply', 'Ambivalent', 'Ambivalent', 'Ambivalent', 'Ambivalent', 'Ambivalent']
Total clarity preds: 308
Lines in prediction file: 308
updating: prediction (deflated 95%)
Archive:  submission.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
     3623  2025-12-09 19:18   prediction
---------                     -------
     3623                     1 file


In [ ]:
from google.colab import files
files.download("submission.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>